# Script NER/REL/LLM/NLM con spacy

##### 1. import pdf plumber para importar todo el texto limpio

In [8]:
#pdf plumber es wraper de pdfminer
import pdfplumber

pdf_file_path = "Informe-Rettig-tomo1.pdf"

def extract_text_from_pdf(pdf_file_path, start_page, end_page):
    with pdfplumber.open(pdf_file_path) as pdf:
        text = ""
        for page_num in range(start_page - 1, end_page):  # Adjusting to 0-based index
            page = pdf.pages[page_num]
            text += page.extract_text()
    return text


# Extract text from pages 110 to 210
# solo la rm, para dividir entre "RM" y "el resto"
start_page = 139
end_page = 252
pdf_text = extract_text_from_pdf(pdf_file_path, start_page, end_page)



#### 2. spacy first run

In [6]:
import spacy
# Load the SpaCy English model
nlp = spacy.load("es_core_news_sm")

# Function to process text with SpaCy and extract named entities
def extract_named_entities(text):
    doc = nlp(text)
    named_entities = []
    for ent in doc.ents:
        named_entities.append((ent.text, ent.label_))
    return named_entities

# Process text with SpaCy and extract named entities
named_entities = extract_named_entities(pdf_text)

# Print the extracted named entities
for entity, label in named_entities:
    print(f"Entity: {entity}, Label: {label}")


Entity: detención, Label: LOC
Entity: Región Metropolitana, Label: LOC
Entity: Londres, Label: LOC
Entity: Partido Socialista, Label: ORG
Entity: DINA, Label: ORG
Entity: calle Londres, Label: LOC
Entity: habían, Label: PER
Entity: Remodelación San Borja, Label: LOC
Entity: Santiago, Label: LOC
Entity: Casa de la Cultura de Barrancas, Label: LOC
Entity: Del mismo modo, Label: MISC
Entity: Partido Comunista, Label: ORG
Entity: Leopardo, Label: PER
Entity: El recinto denominado, Label: MISC
Entity: Parque Cerrillos, Label: LOC
Entity: Feria, Label: LOC
Entity: Internacional de Santiago, Label: LOC
Entity: FISA, Label: ORG
Entity: detención, Label: LOC
Entity: Ejército, Label: MISC
Entity: También, Label: LOC
Entity: detención, Label: LOC
Entity: Carabineros, Label: LOC
Entity: Región, Label: LOC
Entity: Región, Label: LOC
Entity: Tejas Verdes, Label: ORG
Entity: Escuela de Ingenieros Militares, Label: LOC
Entity: Escuela de Artillería de Linares, Label: LOC
Entity: Casos de graves violac

In [94]:
# Iterate through the data and print rows where var1P is "PER"
for row in named_entities:
    if row[1] == "LOC":
        print(row[0])

Región
Región Metropolitana
Visión General
La Región Metropolitana
Provincias de Santiago
Chacabuco
Cordillera
Maipo
Melipilla
Talagante
Poder Ejecutivo
Palacio de La Moneda
Gobierno
Región
Estado
Región Metropolitana
Palacio de Gobierno
Valparaíso
Palacio de La
Moneda
Regimiento Blindado Nº
Palacio de La Moneda
Ejército
Santiago
La Moneda
Edecanes
Palacio de La Moneda
Chile
Palacio de
La Moneda
Palacio
Palacio de Gobierno
Región Metropolitana
Región
Palacio de Gobierno
Gobierno Militar
Estado de situación del país
C.O.FF.AA
Región
Región
Estado
C.O.FF.AA
Palacio de La Moneda
Región
Jefe de Zona
Estado de Sitio
Comisión
Base Aérea El Bosque
Ejército
Región
Regimiento
Santiago
Carabineros
Región
Paine
Gobierno
Gobierno
Mención
Estado
Estado
Estado
Palacio de La Moneda
Región
Hospital
San Juan de Dios
Sumar
Aerolite
Elecmetal
Mademsa
La Bandera
La Legua
Estados de Situación
C.O.FF.AA
Polpaico
Palacio de Bellas Artes
Remodelación
San Borja
detención
General San Martín
Los Andes
Valparaíso